<a href="https://colab.research.google.com/github/Haseeb55859/Artificial-intelligence-AI-/blob/main/NOTEBOOK/PHASE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use a clean mount point**

In [12]:
from google.colab import drive
drive.mount('/content/drive_fresh')


Mounted at /content/drive_fresh


**Set Dataset Paths (Skin Cancer 9-class)**

In [13]:
import os

# Base folder for skin cancer dataset
base_dir = "/content/drive_fresh/MyDrive/datasets/skin_cancer9"

print("Contents of base_dir:")
for f in os.listdir(base_dir):
    print(repr(f))   # repr() se spaces/special characters bhi clearly dikhenge


Contents of base_dir:
'train'
'test'
'val'
'manifest.csv'


**ImageDataGenerator**

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- Parameters ---
IMG_HEIGHT = 224
IMG_WIDTH  = 224
BATCH_SIZE = 32
SEED = 42

# --- Directories ---
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")
test_dir  = os.path.join(base_dir, "test")

# -------------------------------
# 1️⃣ Data generators

# Training generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Validation generator (only rescale)
val_datagen = ImageDataGenerator(rescale=1./255)

# Test generator (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)

# -------------------------------
# 2️⃣ Create generators

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # MULTI-CLASS (9 classes)
    shuffle=True,
    seed=SEED
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)


Found 1907 images belonging to 9 classes.
Found 332 images belonging to 9 classes.
Found 118 images belonging to 9 classes.


**Build CNN Model for 9-class Skin Cancer Dataset**

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# -------------------------------
# Build Medium CNN for 9 classes
def build_skin_cnn(input_shape=(224,224,3), num_classes=9):
    model = Sequential([

        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Block 2
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Block 3
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Block 4 (optional deeper)
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        # Dense layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),

        Dense(num_classes, activation='softmax')  # Multi-class output
    ])
    return model

# -------------------------------
# Create model
model = build_skin_cnn()
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    12,845,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,237,961 (50.50 MB)

 Trainable params: 13,237,001 (50.50 MB)

 Non-trainable params: 960 (3.75 KB)

**Compile Model**

In [16]:
from tensorflow.keras.optimizers import Adam

# -------------------------------
# STEP 5 — Compile Model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',  # 9-class classification
    metrics=['accuracy']
)

print("Model compiled successfully.")


Model compiled successfully.


**CallBacks**

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os

# Checkpoint path (model save hoga yaha)
checkpoint_path = "/content/drive_fresh/MyDrive/skin_cancer_models/best_skin_cnn.h5"
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-7)
]

print("Callbacks ready. Model checkpoint path:", checkpoint_path)


Callbacks ready. Model checkpoint path: /content/drive_fresh/MyDrive/skin_cancer_models/best_skin_cnn.h5


** Training Model**

In [18]:
# -------------------------------
# STEP 7 — Train the model

EPOCHS = 50  # aap 50 epochs rakh sakte ho

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.3092 - loss: 3.3279 

60/60 ━━━━━━━━━━━━━━━━━━━━ 1598s 27s/step - accuracy: 0.3096 - loss: 3.3143 - val_accuracy: 0.0602 - val_loss: 3.5742 - learning_rate: 1.0000e-04
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 50s 838ms/step - accuracy: 0.3614 - loss: 1.7153 - val_accuracy: 0.0602 - val_loss: 4.2811 - learning_rate: 1.0000e-04
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 45s 747ms/step - accuracy: 0.4152 - loss: 1.6127 - val_accuracy: 0.0602 - val_loss: 8.6890 - learning_rate: 1.0000e-04
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 46s 767ms/step - accuracy: 0.4462 - loss: 1.5800 - val_accuracy: 0.0633 - val_loss: 8.3807 - learning_rate: 1.0000e-04
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 45s 756ms/step - accuracy: 0.4678 - loss: 1.5428 - val_accuracy: 0.0693 - val_loss: 8.4860 - learning_rate: 5.0000e-05
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 45s 750ms/step - accuracy: 0.4832 - loss: 1.4329 - val_accuracy: 0.1355 - val_loss: 7.0589 - learning_rate: 5.0000e-05
